# Sales Copilot

Additional concepts:

- Copilot determines intent. Either goes to load balance endpoint or Assistant depending on user intent.
- Thread management for multiple users
  - Using shelves to store the user threads
  - Thread will be created or reused as different users access the assistant
- Relationships in files

## Diagram

- **Note**: Not implemented, promptflow and weather services

![Architecture Diagram](../images/copilot-with-assistant.png)

### Load the required libraries

In [ ]:
import os
import sales_agent
from pathlib import Path

from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types import FileObject


### Load the environment variables

In [ ]:
load_dotenv()
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")
email_URI = os.getenv("EMAIL_URI")

### Create the AzureOpenAI clients

In [ ]:
load_balanced_client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

In [ ]:
sales = sales_assistant.get_sales_assistant()

In [ ]:
def semantic_intent(prompt:str)->str:
    prompt_template = """system:
You are an agent that can determine intent from the following list of intents and return the intent that best matches the user's question.
 
List of intents:
SalesAgent: questions or operations related to calculations, customers, sales, orders, inventory, items, and sellers.
OtherAgent: any other question

user:
<QUESTION>

Output in ONE word."""


    full_prompt = prompt_template.replace("<QUESTION>", prompt)
    completion = load_balanced_client.chat.completions.create(
            model=api_deployment_name,
            messages=[
                {
                    "role": "user",
                    "content": full_prompt,
                },
            ],
            max_tokens=2,
            temperature=0.1           
        )    
    try:
        return completion.choices[0].message.content
    except:
        return "Unknown"
    

### Go to load balance endpoint or to the Assistant based on intent

In [ ]:
def process_for_intent(user: str, user_id: str, prompt: str):
    intent = semantic_intent(prompt)
    print(f"Intent: {intent}")
    
    if intent=="SalesAgent":
        sales.process(user, user_id, "What customers are in Florida?")
    elif intent=="OtherAgent":
        completion = load_balanced_client.chat.completions.create(
            model=api_deployment_name,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ]
        )
        print(completion.choices[0].message.content)
        
    else:        
        print("Unknown Intent")

### Have a conversation with the Assistant

In [ ]:
process_for_intent("Suzan", "user_123", "What is the speed of light?")

In [ ]:
process_for_intent("John", "user_123", "What customers are in Florida?")

In [ ]:
process_for_intent("Mary", "user_234", "What seller has had the most sales?")

In [ ]:
process_for_intent("John", "user_123", "What is the most sold product?")

In [ ]:
process_for_intent("Mary", "user_234", "Chart product sales by State.")

### Cleanup

In [ ]:
sales.cleanup()